In [1]:
from cnns.core import LasagneGoogLeNetInceptionV3 as inceptv3
import lasagne
import numpy as np
import pickle as pkl
import theano
import theano.tensor as T
import skimage.io as io
from cnns.utils import cnn_utils as cu
import sys
import os
sys.path.insert(1, os.environ['CAFFE_HOME'] + '/python')
import caffe
import urllib2
from skimage import img_as_float
from PIL import Image
import StringIO
from cnns.utils import imagenet_utils as imu
import csv

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
def convert_dicts_to_sorted_lists(ground_truth_cats, pred_cats, k=1):
    assert len(ground_truth_cats) == len(pred_cats)
    pred_cats_list = [None]*len(pred_cats)
    ground_truth_cats_list = [None]*len(ground_truth_cats)
    
    for idx, pred_cat in pred_cats.iteritems():
        pred_cats_list[idx-1] = pred_cat
    for idx, ground_truth_cat in ground_truth_cats.iteritems():
        ground_truth_cats_list[idx-1] = ground_truth_cat
    
    sorted_pred_cats_list = [sorted(pred_cat, key=lambda x: x[1], reverse=True) for pred_cat in pred_cats_list]
    return sorted_pred_cats_list, ground_truth_cats_list

def compute_top_k_acc(ground_truth_cats_list, sorted_pred_cats_list, k=1):
    acc =  np.mean([ground_truth_cat in set([p[0] for p in pred_cat[:k]])
                    for ground_truth_cat, pred_cat in zip(ground_truth_cats_list, sorted_pred_cats_list)])
    return acc


In [5]:
cnns_dir = '/Users/babasarala/repos/cnns'
evaluation_dir = '/Users/babasarala/Desktop/evaluation'
image_url_files_dir = os.path.join(evaluation_dir, 'image_url_files')
mat_filepath = '/Users/babasarala/Downloads/ILSVRC2013_devkit/data/meta_clsloc.mat'
ground_truth_filepath = os.path.join(evaluation_dir, 'raw_images/ILSVRC2013_clsloc_validation_ground_truth.txt')
prune_filepath = os.path.join(cnns_dir, 'config/image_net_categories_animals_screenshots_devices_2016_05_04.json')

In [6]:
param_pkl_filepath = '/Users/babasarala/repos/cnns/models/inception_v3.pkl'
model = inceptv3.LasagneGoogLeNetInceptionV3(param_pkl_filepath)
model.define_classification_flow(layer_name='pool3')

In [17]:
model.net.keys()

['pool_1',
 'conv',
 'mixed_6/join',
 'mixed_1/join',
 'softmax',
 'mixed_4/join',
 'mixed_2/join',
 'mixed_3/join',
 'mixed_5/join',
 'mixed_9/join',
 'pool3',
 'mixed/join',
 'pool',
 'conv_4',
 'mixed_8/join',
 'input',
 'mixed_7/join',
 'mixed_10/join',
 'conv_1',
 'conv_3',
 'conv_2']

In [9]:
from lasagne.layers import get_output_shape
get_output_shape(model.net['mixed_10/join'])

(None, 2048, 8, 8)

In [7]:
img_filepath = '/Users/babasarala/Desktop/evaluation/raw_images/ILSVRC2012_img_val/ILSVRC2012_val_00000001.JPEG'

In [12]:
img_filepath_2 = '/Users/babasarala/Desktop/evaluation/raw_images/ILSVRC2012_img_val/ILSVRC2012_val_00033957.JPEG'

In [13]:
all_params = pkl.load(open(param_pkl_filepath, 'rb'))

In [14]:
labels = model.labels[1:1001]
assert '' not in labels

In [13]:
prob = model.compute_category_probability(cu.load_image(img_filepath, model_lib='lasagne'))
print prob[:5]

[  9.78308017e-05   2.12735526e-04   9.22015824e-05   1.28104220e-04
   1.71408668e-04]


In [10]:
print len(model.int_output)

2048


In [16]:
model.compute_intermediate_output(cu.load_image(img_filepath, model_type='lasagne'), 'pool3')

array([ 0.00758328,  0.29427242,  0.34122628, ...,  0.06805431,
        0.00977449,  0.18829867], dtype=float32)

In [26]:
tfm_fcn = cu.get_tfm_fcn_from_json(prune_filepath)

In [19]:
tfm_prob = cu.combine_probabilities([(tfm_fcn(p[0]), p[1]) for p in prob])

In [20]:
tfm_prob

[(u'wildlife', 0.92914776529789112),
 (u'other', 0.05484376608820795),
 (u'screenshots', 0.00013468083923863679),
 (u'devices', 0.0016641869450077446),
 (u'furry_friends', 0.013655060609820278)]

# Sample Validation set and report results

In [21]:
ground_truth_labels = imu.load_validation_ground_truth(ground_truth_filepath, mat_filepath)

In [36]:
def load_image(img_filepath=None, url=None, src='local'):
    try:
        if src == "url":
            assert url is not None
            url_response = urllib2.urlopen(url)
            image = np.asarray(Image.open(StringIO.StringIO(url_response.read())))
        elif src == "local":
            assert img_filepath is not None
            image = io.imread(curr_img_filepath)
            if image.ndim == 2:
                image = image[:,:,np.newaxis]
                return np.repeat(image, 3, axis=2)
            return image
    except:
        print "Image was not found or is not in the right format! No data has been entered"
        return None

In [41]:
num_validation_imgs = 50000
img_idxs = [f+1 for f in np.random.permutation(num_validation_imgs)[:num_sample]]
pred_labels = ['']*num_sample
pruned_pred_labels = ['']*num_sample
for i, f in enumerate(img_idxs):
    if (i+1)%100 == 0:
        print '%i images processed'%(i+1)
    num_digits = len(str(f))
    curr_img_filepath = os.path.join(evaluation_dir,
                                 'raw_images/ILSVRC2012_img_val/ILSVRC2012_val_000' + \
                                 '0'*(5-num_digits) + '%i.JPEG'%f)
    curr_img = cu.load_image(curr_img_filepath, model_type='lasagne')
    pred_labels[i] = model.compute_category_probability(curr_img)
    #pruned_pred_labels[i] = cu.combine_probabilities([(tfm_fcn(pred_label[0]), pred_label[1]) \
    #                                                      for pred_label in pred_labels[i]])

100 images processed
200 images processed
300 images processed
400 images processed
500 images processed
600 images processed
700 images processed
800 images processed
900 images processed
1000 images processed


In [42]:
ground_truth_sample_labels = [ground_truth_labels[img_idx] for img_idx in img_idxs]

In [43]:
p = compute_top_k_acc(ground_truth_sample_labels, pred_labels, k=1)
print 'Top-1 Accuracy rate: %.2f%%'%(100.*p)
p = compute_top_k_acc(ground_truth_sample_labels, pred_labels, k=5)
print 'Top-5 Accuracy rate: %.2f%%'%(100.*p)
print '...holy shit!'

Top-1 Accuracy rate: 77.10%
Top-5 Accuracy rate: 92.30%
...holy shit!


# Full Validation set and report results

In [90]:
# ILSVRC2012 Validation Set

In [88]:
ground_truth_cats = imu.load_validation_ground_truth(ground_truth_filepath, mat_filepath)
results_dirpath = '%s/results/ILSVRC2012_val/test_dir'%(evaluation_dir)
pred_cats = cu.combine_checkpoint_results(results_dirpath)

In [33]:
ground_truth_cats = imu.load_validation_ground_truth(ground_truth_filepath, mat_filepath)
ground_truth_collapsed_cats = {img_id: tfm_fcn(category) for img_id, category in ground_truth_cats.iteritems()}
results_dirpath = '%s/results/ILSVRC2012_val/GoogLeNetv3_collapsed_2'%(evaluation_dir)
pred_cats = cu.combine_checkpoint_results(results_dirpath)

In [60]:
# print out classification report
sorted_pred_cats_list, ground_truth_cats_list = convert_dicts_to_sorted_lists(ground_truth_collapsed_cats, 
                                                                              pred_cats, 
                                                                              k=1)
top_pred_cats_list = [sorted_pred_cat[0][0] for sorted_pred_cat in sorted_pred_cats_list]
print(classification_report(ground_truth_cats_list, top_pred_cats_list))

             precision    recall  f1-score   support

    devices       0.88      0.78      0.83      1000
furry_friends       0.98      0.98      0.98      6250
      other       0.98      0.99      0.98     29100
screenshots       0.80      0.96      0.87        50
   wildlife       0.99      0.98      0.98     13600

avg / total       0.98      0.98      0.98     50000



In [85]:
# Productivity-based classifiers - this segment collects all the "<mem_id>_true_<category>.csv" files and combines
# them into a single, much more manageable csv file with the image url and category

mem_ids = [5, 8, 1496616, 1946418, 2028464, 2685009, 3473194, 3928074]
categories = ['documents', 'whiteboards', 'sketches', 'other']
productivity_csv_filepath = '%s/image_url_files/productivity/csv/productivity_ground_truth.csv'%(evaluation_dir)
with open(productivity_csv_filepath, 'a+') as g:
    writer = csv.writer(g)
    writer.writerow(['image_url', 'category'])
    for mem_id in mem_ids:
        for category in categories:
            true_filepath = '%s/image_url_files/productivity/csv/true/%i_true_%s.csv' \
                            %(evaluation_dir, mem_id, category)
            urls = []
            with open(true_filepath, 'rb') as f:
                reader = csv.reader(f)
                _ = reader.next() # header
                auth_token = cu.get_auth_token(mem_id)
                urls = [['http://download-dot-maestro-prod.appspot.com/%i?auth_token=%s' \
                         % (int(row[0]), auth_token), category] for row in reader]
                if len(urls) > 0:
                    writer.writerows(urls)          

[u'wildlife',
 u'other',
 u'furry_friends',
 u'other',
 u'other',
 u'wildlife',
 u'wildlife',
 u'other',
 u'other',
 u'wildlife',
 u'wildlife',
 u'furry_friends',
 u'wildlife',
 u'other',
 u'other',
 u'wildlife',
 u'wildlife',
 u'wildlife',
 u'other',
 u'other',
 u'wildlife',
 u'furry_friends',
 u'other',
 u'other',
 u'wildlife',
 u'other',
 u'wildlife',
 u'furry_friends',
 u'wildlife',
 u'other',
 u'wildlife',
 u'other',
 u'wildlife',
 u'other',
 u'other',
 u'other',
 u'wildlife',
 u'other',
 u'other',
 u'wildlife',
 u'other',
 u'other',
 u'wildlife',
 u'furry_friends',
 u'furry_friends',
 u'other',
 u'other',
 u'furry_friends',
 u'other',
 u'wildlife',
 u'wildlife',
 u'wildlife',
 u'other',
 u'other',
 u'wildlife',
 u'other',
 u'furry_friends',
 u'other',
 u'other',
 u'other',
 u'other',
 u'furry_friends',
 u'wildlife',
 u'wildlife',
 u'other',
 u'wildlife',
 u'wildlife',
 u'other',
 u'other',
 u'other',
 u'other',
 u'wildlife',
 u'wildlife',
 u'other',
 u'wildlife',
 u'other',
 u'fu